### Get googld cloud project id 

In [ ]:
!gcloud config list project

### Set googld cloud config 

In [ ]:
gcloud config set compute/region $REGION
gcloud config set ai_platform/region global

### Create bucket

In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}; then
    gsutil mb -l ${REGION} gs://${BUCKET}
fi

### Usage of argparse

In [ ]:
# https://docs.python.org/ko/3/library/argparse.html

import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    name or flags...  # ex: "--job-dir"
    [, nargs]  # ex: "+" 존재하는 모든 명령행 인자를 리스트로 모읍니다. 또한, 적어도 하나의 명령행 인자가 제공되지 않으면 에러 메시지가 만들어집니다
    [, default]
    [, type]  # ex: int, float
    [, required]  # ex: True or False
    [, help]
)

args = parser.parse_args()
arguments = args.__dict__


### train_and_evaluate

1. build_wide_deep_model
  - create input layers
    - {  name: keras.Input(name, shape, dtype)  }
  - create feature columns of wide and deep features
    - create numeric_column(name)
    - create categorical column
      - categorical_column_with_vocabulary_list(name, values)
      - indicator_column(category)
    - create new deep feature with numeric columns
      - bucketized_column(fc, boundaries=list)
      - indicator_column(bucketized): one-hot encoding
      - crossed_column([bucketized1, bucketized2], hash_bucket_size)
      - embedding_column(crossed, dimension=nembeds)
  - stack DenseFeatures layers of wide and deep features
  - add Dense layers
    - activation=relu
  - add Concatenate layer of wide and deep layers
  - add output Dense layer
    - units=1, activation=linear
  - create model with inputs and outputs
  - compile model
    - loss=mse, optimizer=adam, metrics=[rmse, mse]
2. load_dataset: traind, evals
  - get dataset from csv
    - make_csv_dataset(pattern, batch_size, COLUMNS, DEFAULTS)
  - append map function to split feature and labels 
  - if train, then shuffle and repeat
    - repeat(None): infinite repeat
    - shuffle(buffer_size=1000): input proper value
  - prefetch
    - buffer_size=1 is AUTOTUNE
3. model.fit
  - trainds
  - validation_data=evalds
  - epochs = NUM_EVALS or NUM_EPOCHS
  - steps_per_epoch = train_examples // (batch_size * epochs)
  - callbacks=[callbacks]
4. set hypertune
  - report_hyperparameter_tuning_metric
    - hyperparameter_metric_tag='rmse'  # monitoring target name
    - metric_value=history.history['val_rmse'][-1],  # monitoring target value: 
    - global_step=args['num_epochs']
5. save model

### steps_per_epoch 의미

steps_per_epoch = train_examples // (batch_size * epochs)
- virtual epoch. 1개의 epoch을 여러 step으로 나누고 각 step이 epoch과 동일하게 eval, callbacks 처리됨

### Deleting and deploying model

In [ ]:
%%bash
MODEL_NAME="babyweight"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION='gs://qwiklabs-gcp-00-0db9b1bc58c6/babyweight/trained_model/20210602015319'
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION"

# 모델 삭제 전에 모든 버전을 삭제해야 함
# default 버전은 다른 버전을 모두 삭제한 후에 삭제 가능함
gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME} -q
gcloud ai-platform models delete ${MODEL_NAME} -q

# 모델 생성 후 버전 생성
gcloud ai-platform models create ${MODEL_NAME} --regions ${REGION}
gcloud ai-platform versions create ${MODEL_VERSION} \
    --model=${MODEL_NAME} \
    --origin=${MODEL_LOCATION} \
    --runtime-version=2.1 \
    --python-version=3.7